In [1]:
from scipy.io import loadmat
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
train_raw = loadmat('./data/train_32x32.mat')
test_raw = loadmat('./data/test_32x32.mat')

In [3]:
train_images = np.array(train_raw['X'])
test_images = np.array(test_raw['X'])

train_labels = train_raw['y']
test_labels = test_raw['y']

In [4]:
# train_labels[train_labels == 10] = 0
# test_labels[test_labels == 10] = 0

In [5]:
train_labels = train_labels - 1
test_labels = test_labels - 1

In [6]:
train_images = np.moveaxis(train_images, -1, 0)
test_images = np.moveaxis(test_images, -1, 0)

print(train_images.shape)
print(test_images.shape)

(73257, 32, 32, 3)
(26032, 32, 32, 3)


In [7]:
train_images = train_images.astype('float')
test_images = test_images.astype('float')
train_labels = train_labels.astype('int')
test_labels = test_labels.astype('int')
train_images /= 255.0
test_images /= 255.0

In [8]:
tf.random.set_seed(126)

image_generator = ImageDataGenerator(
            rotation_range=10,
            zoom_range=0.10,
            shear_range=0.5,
            width_shift_range=0.10,
            height_shift_range=0.10,
            horizontal_flip=True,
            vertical_flip=False)

augment_size = 30000

randidx = np.random.randint(train_images.shape[0], size=augment_size)
x_augmented = train_images[randidx].copy()
y_augmented = train_labels[randidx].copy()
x_augmented = image_generator.flow(x_augmented, np.zeros(augment_size),
                                   batch_size=augment_size, shuffle=False).next()[0]

train_images = np.concatenate((train_images, x_augmented))
train_labels = np.concatenate((train_labels, y_augmented))

In [9]:
print(train_images.shape)
print(train_labels.shape)

(103257, 32, 32, 3)
(103257, 1)


In [10]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(input_shape=(32, 32, 3), kernel_size=(3,3), filters=32, padding='same', activation='relu'),
    tf.keras.layers.Conv2D(kernel_size=(3,3), filters=64, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Conv2D(kernel_size=(3,3), filters=128, padding='same', activation='relu'),
    tf.keras.layers.Conv2D(kernel_size=(3,3), filters=256, padding='valid', activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(units=10, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])


In [11]:
history = model.fit(train_images, train_labels, epochs=25, validation_split=0.25)

Epoch 1/25
2421/2421 [==============================] - 22s 8ms/step - loss: 2.2411 - accuracy: 0.1887 - val_loss: 2.2366 - val_accuracy: 0.1917
Epoch 2/25
2421/2421 [==============================] - 19s 8ms/step - loss: 2.2382 - accuracy: 0.1893 - val_loss: 2.2366 - val_accuracy: 0.1917
Epoch 3/25
1133/2421 [=============>................] - ETA: 9s - loss: 2.2372 - accuracy: 0.1904

KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], 'g-', label='accuracy')
plt.plot(history.history['val_accuracy'], 'k--', label='val_accuracy')
plt.xlabel('Epoch')
plt.ylim(0.7, 1)
plt.legend()

plt.show()

model.evaluate(test_images, test_labels, verbose=0)